# Housing Prices Prediction 
In this notebook, I intend to use the <a href="https://www.kaggle.com/competitions/home-data-for-ml-course/overview">Housing Prices Competiton</a> from Kaggle to apply data processing, exploratory data analysis, and finally use regression to predict sales prices for houses. 

In [1]:
import pandas as pd
import numpy as np

# Data Visualization 
import seaborn as sns
import matplotlib.pyplot as plt

## Initial Data Cleaning

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### Duplicates and empty rows

In [3]:
# Remove all duplicate rows, rows all empty fields

train_df = train_df.drop_duplicates()
train_df = train_df.dropna(how="all")

# Remove rows if target is missing
train_df = train_df.dropna(subset=['SalePrice'])


### Missing values

In [ ]:
# Identify which columns have missing values
missing = train_df.isna().sum()[train_df.isna().sum().values > 0].sort_values(ascending=False)
print(missing)

# Index of columns with missing values 
print(missing.index)

# Categorical Variables with missing values 
missing_categorical = train_df[missing.index].select_dtypes(include=["object"])
print(missing_categorical)

# Numerical Variables with missing values 
missing_numerical = train_df[missing.index].select_dtypes(exclude=["object"])
print(missing_numerical)


PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
Electrical         1
dtype: int64
Index(['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'MasVnrType', 'FireplaceQu',
       'LotFrontage', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageQual', 'GarageCond', 'BsmtFinType2', 'BsmtExposure',
       'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrArea', 'Electrical'],
      dtype='object')
     PoolQC MiscFeature Alley  Fence MasVnrType FireplaceQu GarageType  \
0       NaN         NaN   NaN    NaN    BrkFace         NaN     Attchd   
1       NaN         NaN   NaN    NaN        NaN          TA     Attchd   
2       NaN         NaN   NaN    NaN    BrkFace        

Large number of missing values for categorical variables, indicates that houses do not have these features, we will replace with 'NA'.
Only three numerical variables have missing values, we will replace the missing values with 0. 

In [12]:
# Replace NA values for categorical variables
categorical = train_df.select_dtypes(include=["object"])
train_df[categorical.columns] = categorical.fillna("NA") 

# Replace NaN values for numerical variables 
numerical = train_df.select_dtypes(exclude=["object"])
train_df[numerical.columns] = numerical.fillna(0)

missing = train_df.isna().sum()[train_df.isna().sum().values > 0].sort_values(ascending=False)
print(missing)


Series([], dtype: int64)


## Data Summary and Visualization

In [14]:
def summarise_data(df):
    summary = {
        "Feature": [],
        "Data Type": [],
        "Missing Values": [],
        "Min": [],
        "Max": [],
        "Mean": [],
        "Std": [],
        "Lower Quartile": [],
        "Median": [],
        "Upper Quartile": [],
        "Mode": []
    }

    summary_stats = df.describe().T

    # Data Types and Summary Statistics
    for column in df.columns: 
        summary["Feature"].append(column)
        summary["Data Type"].append(df[column].dtype)

        summary["Missing Values"].append(df[column].isnull().sum())

        if df[column].dtype in [np.int64, np.float64]:
            summary["Min"].append(summary_stats.loc[column, "min"])
            summary["Max"].append(summary_stats.loc[column, "max"])
            summary["Mean"].append(summary_stats.loc[column, "mean"])
            summary["Std"].append(summary_stats.loc[column, "std"])
            summary["Lower Quartile"].append(summary_stats.loc[column, "25%"])
            summary["Median"].append(summary_stats.loc[column, "50%"])
            summary["Upper Quartile"].append(summary_stats.loc[column, "75%"])

            summary["Mode"].append(df[column].mode()[0])
        
        else: 
            summary["Min"].append("-")
            summary["Max"].append("-")
            summary["Mean"].append("-")
            summary["Std"].append("-")
            summary["Lower Quartile"].append("-")
            summary["Median"].append("-")
            summary["Upper Quartile"].append("-")

            summary["Mode"].append(df[column].mode()[0])

    
    return pd.DataFrame(summary)

summarise_data(train_df)

,Feature,Data Type,Missing Values,Min,Max,Mean,Std,Lower Quartile,Median,Upper Quartile,Mode
0,Id,int64,0,1.0,1460.0,730.5,421.610009,365.75,730.5,1095.25,1
1,MSSubClass,int64,0,20.0,190.0,56.89726,42.300571,20.0,50.0,70.0,20
2,MSZoning,object,0,-,-,-,-,-,-,-,RL
3,LotFrontage,float64,0,0.0,313.0,57.623288,34.664304,42.0,63.0,79.0,0.0
4,LotArea,int64,0,1300.0,215245.0,10516.828082,9981.264932,7553.5,9478.5,11601.5,7200
...,...,...,...,...,...,...,...,...,...,...,...
76,MoSold,int64,0,1.0,12.0,6.321918,2.703626,5.0,6.0,8.0,6
77,YrSold,int64,0,2006.0,2010.0,2007.815753,1.328095,2007.0,2008.0,2009.0,2009
78,SaleType,object,0,-,-,-,-,-,-,-,WD
79,SaleCondition,object,0,-,-,-,-,-,-,-,Normal


In [8]:
# MSSubClass should be a categorical variable
train_df["MSSubClass"] = train_df["MSSubClass"].astype("object")

train_df["MSSubClass"].value_counts()


MSSubClass
20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: count, dtype: int64